In [131]:
import requests
import pandas as pd
import json
import time

In [32]:
apiKey = "IaF6zOUaj8VG4ambmUwOodEFaxBnrR3A"
apiUrl = "https://aeroapi.flightaware.com/aeroapi/"
auth_header = {'x-apikey':apiKey}

# GET flights from DMK airport

In [33]:
Airport_id = 'DMK'
Airport_start = '2022-12-01T00:00:00Z'
Airport_end = '2022-12-04T00:00:00Z'

# Y-M-D
# 1) 2022-12-01 - 2022-12-04


In [73]:
#----- get flight data from DMK airport -----#

# response = requests.get(apiUrl + f"airports/{Airport_id}/flights?start=2022-12-01T00%3A00%3A00Z&end=2022-12-04T00%3A00%3A00Z",
#  headers=auth_header)

# if response.status_code == 200:
#     print(response.json())
# else:
#     print("Error executing request")

In [83]:
print(json.dumps(response.json(), indent=4))

{
    "arrivals": [
        {
            "ident": "TLM219",
            "ident_icao": "TLM219",
            "ident_iata": "SL219",
            "fa_flight_id": "TLM219-1669920000-schedule-0938",
            "operator": "TLM",
            "operator_icao": "TLM",
            "operator_iata": "SL",
            "flight_number": "219",
            "registration": "HS-LGI",
            "atc_ident": null,
            "inbound_fa_flight_id": "TLM218-1669899290-airline-0078",
            "codeshares": [],
            "codeshares_iata": [],
            "blocked": false,
            "diverted": false,
            "cancelled": false,
            "position_only": false,
            "origin": {
                "code": "VABB",
                "code_icao": "VABB",
                "code_iata": "BOM",
                "code_lid": null,
                "timezone": "Asia/Kolkata",
                "name": "Chatrapati Shivaji Int'l",
                "city": "Mumbai",
                "airport_info_url": "/air

In [118]:
Arrivals_FlightID = []
Departures_FlightID = []

In [119]:
for i in response.json()['arrivals']:
    Arrivals_FlightID.append(i['fa_flight_id'])
    
for i in response.json()['departures']:
    Departures_FlightID.append(i['fa_flight_id'])

In [121]:
for item in Departures_FlightID:
    print(item)

AIQ642-1669939380-schedule-0476
NOK340-1669938420-schedule-0407
AIQ3437-1669938420-schedule-0400
AIQ311-1669938420-schedule-0399
AIQ3213-1669938420-schedule-0403
NOK122-1669938420-schedule-0392
NOK900-1669937520-schedule-0590
AIQ3102-1669937520-schedule-0604
AIQ3188-1669938420-schedule-0397
AIQ3029-1669937520-schedule-0599
TLM702-1669937520-schedule-0574
TLM506-1669936380-schedule-0442
NOK100-1669936380-schedule-0453
AIQ3252-1669936380-schedule-0445
AIQ396-1669936380-schedule-0404


# GET data from flight ID

In [132]:
# i = 1
# for id in Departures_FlightID:
#     response = requests.get(apiUrl + f"flights/{id}/track",
#     headers=auth_header)

#     if response.status_code == 200:
#         print(i)
#         # print(response.json())
#         df = pd.DataFrame.from_dict(response.json()['positions'])
#         df['fa_flight_id'] = id
#         df.to_csv(f'flights_data/{id}.csv',index = False)
            
#     else:
#         print("Error executing request")
        
#     i+=1
#     time.sleep(7)

1
2
Error executing request
Error executing request
Error executing request
6
7
8
9
10
11
12
13
14
15


In [123]:
# import json

print(json.dumps(response.json(), indent=4))

{
    "positions": [
        {
            "fa_flight_id": null,
            "altitude": 4,
            "altitude_change": "C",
            "groundspeed": 179,
            "heading": 208,
            "latitude": 13.89944,
            "longitude": 100.59736,
            "timestamp": "2022-12-03T23:57:52Z",
            "update_type": "A"
        },
        {
            "fa_flight_id": null,
            "altitude": 10,
            "altitude_change": "C",
            "groundspeed": 184,
            "heading": 208,
            "latitude": 13.88798,
            "longitude": 100.59111,
            "timestamp": "2022-12-03T23:58:08Z",
            "update_type": "A"
        },
        {
            "fa_flight_id": null,
            "altitude": 17,
            "altitude_change": "C",
            "groundspeed": 186,
            "heading": 208,
            "latitude": 13.87573,
            "longitude": 100.58441,
            "timestamp": "2022-12-03T23:58:24Z",
            "update_type": "A"
    

In [127]:
df = pd.read_csv(f'flights_data/{id}.csv')
df.head()

,fa_flight_id,altitude,altitude_change,groundspeed,heading,latitude,longitude,timestamp,update_type
0,AIQ642-1669939380-schedule-0476,4,C,179,208,13.89944,100.59736,2022-12-03T23:57:52Z,A
1,AIQ642-1669939380-schedule-0476,10,C,184,208,13.88798,100.59111,2022-12-03T23:58:08Z,A
2,AIQ642-1669939380-schedule-0476,17,C,186,208,13.87573,100.58441,2022-12-03T23:58:24Z,A
3,AIQ642-1669939380-schedule-0476,23,C,190,233,13.86502,100.57491,2022-12-03T23:58:40Z,A
4,AIQ642-1669939380-schedule-0476,28,C,196,276,13.86214,100.56075,2022-12-03T23:58:56Z,A


In [129]:
import time
print(1)
time.sleep(10)
print('after10sec')

1
after10sec
